# **Retriever: Faiss**

## **1. Install and import bibraries**

In [ ]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers
!sudo apt-get install libomp-dev
!pip install faiss-cpu

In [2]:
import numpy as np
import collections
import torch
import faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## **2. Download dataset**

In [3]:
dataset_name = "squad_v2"
raw_datasets = load_dataset(dataset_name, split="train+validation")
raw_datasets

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 142192
})

## **3. Filter out non-answerable samples**

In [4]:
columns = raw_datasets.column_names
columns_to_keep = ['id', 'context', 'question', 'answers']
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
raw_datasets = raw_datasets.remove_columns(columns_to_remove)
raw_datasets

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 142192
})

In [5]:
raw_datasets = raw_datasets.filter(lambda x: len(x["answers"]["text"]) > 0)

Filter:   0%|          | 0/142192 [00:00<?, ? examples/s]

## **4. Intialize pre-trained model**

In [6]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

## **5. Create get vector embedding functions**

In [7]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [8]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    model_output = model(**encoded_input)

    return cls_pooling(model_output)

In [9]:
# Test functionality
embedding = get_embeddings(raw_datasets['question'][0])
embedding.shape

torch.Size([1, 768])

In [10]:
embedding_column = "question_embedding"

embedding_dataset = raw_datasets.map(
    lambda x: {embedding_column: get_embeddings(x["question"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/92749 [00:00<?, ? examples/s]

In [11]:
embedding_dataset.add_faiss_index(column=embedding_column)

  0%|          | 0/93 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'context', 'question', 'answers', 'question_embedding'],
    num_rows: 92749
})

In [15]:
embedding_dataset[0]

{'id': '56be85543aeaaa14008c9063',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]},
 'question_embedding': [0.11258814483880997,
  -0.3660516142845154,
  -0.016732580959796906,
  -0.07673148810863495,
  -0.10467

## **6. Search similar samples with a question**

In [16]:
input_question = "When did Beyonce start becoming popular?"

input_question_embedding = get_embeddings([input_question])
input_question_embedding = input_question_embedding.detach().cpu().numpy()

top_k = 5
scores, samples = embedding_dataset.get_nearest_examples(
    embedding_column, input_question_embedding, k=top_k
)

for idx, score in enumerate(scores):
    print(f"Top {idx + 1}\t Score: {score}")
    print(f"Question: {samples['question'][idx]}")
    print(f"Context: {samples['context'][idx]}")
    print()

Top 1	 Score: 0.0
Question: When did Beyonce start becoming popular?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

Top 2	 Score: 2.613537311553955
Question: When did Beyoncé rise to fame?
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actres

# **Apply the question-to-answer model to answer the question**

In [17]:
# Load model from hub
from transformers import pipeline

pipeline_name = "question-answering"
model_name = "nhutan410/distilbert-finetuned-squadv2"

pipe = pipeline(pipeline_name, model=model_name)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [18]:
print(f"Input question: {input_question}")
for idx, score in enumerate(scores):
    question = samples["question"][idx]
    context = samples["context"][idx]
    answer = pipe(question=question, context=context)

    print(f"Top {idx + 1}\t Score: {score}")
    print(f"Context: {context}")
    print(f"Answer: {answer}")
    print()

Input question: When did Beyonce start becoming popular?
Top 1	 Score: 0.0
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answer: {'score': 0.5068965554237366, 'start': 276, 'end': 286, 'answer': 'late 1990s'}

Top 2	 Score: 2.613537311553955
Context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an Am

In [19]:
test_datasets = load_dataset(dataset_name, split='validation')
test_datasets

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 11873
})

In [23]:
TOP_K = 3
for idx, input_question in enumerate(embedding_dataset['question'][200:210]):
    input_quest_embedding = get_embeddings([input_question]).cpu().detach().numpy()
    scores, samples = embedding_dataset.get_nearest_examples(
        embedding_column, input_quest_embedding, k=TOP_K
    )
    print(f'Question {idx + 1}: {input_question}')
    for jdx, score in enumerate(scores):
        print(f'Top {jdx + 1}\tScore: {score}')
        context = samples['context'][jdx]
        answer = pipe(
            question=input_question,
            context=context
        )
        print(f'Context: {context}')
        print(f'Answer: {answer}')
        print()
    print()

Question 1: How many awards was Beyonce nominated for at the 52nd Grammy Awards?
Top 1	Score: 0.0
Context: At the 52nd Annual Grammy Awards, Beyoncé received ten nominations, including Album of the Year for I Am... Sasha Fierce, Record of the Year for "Halo", and Song of the Year for "Single Ladies (Put a Ring on It)", among others. She tied with Lauryn Hill for most Grammy nominations in a single year by a female artist. In 2010, Beyoncé was featured on Lady Gaga's single "Telephone" and its music video. The song topped the US Pop Songs chart, becoming the sixth number-one for both Beyoncé and Gaga, tying them with Mariah Carey for most number-ones since the Nielsen Top 40 airplay chart launched in 1992. "Telephone" received a Grammy Award nomination for Best Pop Collaboration with Vocals.
Answer: {'score': 0.8159785270690918, 'start': 51, 'end': 54, 'answer': 'ten'}

Top 2	Score: 2.38472580909729
Context: At the 57th Annual Grammy Awards in February 2015, Beyoncé was nominated for si